# Running Algorithms at Scale

In order to run algorithms in the scaled-up cloud compute environment, they must first be "registered" in the Algorithm Catalog. This will make them available to other MAAP users, clearly define their inputs and outputs, and prepare them to be run easily in the Data Processing System (DPS).

![Running Algorithms overview in context diagram](_static/running_algorithms_overview.png)


## Register an Algorithm

### Clone a test algorithm

This is an example algorithm you can use for this getting started guide:
https://github.com/MAAP-Project/dps-unit-test
![Demo GitHub repo](_static/example_alg1.png)

1. Make a new folder for your test algorithm. Open a terminal here (File > New > Terminal or use the blue "+" button above the Jupyter file browser).
![New Folder](_static/clone_demo1.png)

2. Copy the Github clone link from https://github.com/MAAP-Project/dps-unit-test
![Copy .git link](_static/clone_demo2.png)

3. Open the built-in Jupyter Github UI to the left of the file browser. Choose "Clone a Repository" and paste in the .git link you copied from the Github repository.
![Clone a Repository](_static/clone_demo3.png)
![Paste .git link](_static/clone_demo4.png)

4. You should see a new folder created with the repo you cloned. If you browse to that folder and open up the Jupyter Github UI again, it will show you some info about that repo.
![Algorithm folder was created](_static/clone_demo5.png)
![Browse to folder](_static/clone_demo6.png)
![Look at Github UI](_static/clone_demo7.png)

5. If you want to make changes to the code and have your own copy of it to register, Clone the code into a public repository in Github or in MAAP Gitlab.


### Register the algorithm

1. Make sure code is ready and saved -> open the Launcher -> open the Register Algorithm tool in the MAAP Extensions section
![Register Algorithm tool in Launcher](_static/launcher-register-algorithm.png)

2. First you fill in the public code-repository information. The Repository URL is the URL of the .git file. The run and build commands must be the full path of the 
scripts that will be used by the DPS to build and execute the algorithm. Typically these will be the repository_name/script_name.sh, as demonstrated in this screenshot:
![Register Algorithm repository information](_static/register-1-repo.png)

3. Once that is complete "Add General Information". The algorithm name will be the unique identifier for the algorithm in the MAAP system. Disk Space is the minimum amount of space you expect—it gives the DPS an approximation to help optimize the run.

The Container URL is a URL of the Stack you are using as a base for the algorithm. In this example we use:
`https://mas.maap-project.org/root/maap-workspaces/base_images/vanilla:v3.0.1`
![Register Algorithm general information](_static/register-2-general.png)

To find another Container URL, go to:
https://repo.maap-project.org/root/maap-workspaces/container_registry (choose `Packages and Registries > Container registry` if you go to the main maap-workspaces area). Find your base Stack and dig in until you can copy the link of the specific version of Stack that you need, as demonstrated in these screenshots:
![Container registry](_static/registry-1-containers.png)
![Container vanilla](_static/registry-2-vanilla.png)
![Container copy link](_static/registry-3-copy-link.png)

4. Fill in the Input section. There are File Inputs and Positional Inputs (command-line parameters to adjust how the algorithm runs). In our example we have on File Input called `input_file`. For each input you can add a Description, a Default Value, and mark whether it's required or optional.

![Register Algorithm file inputs](_static/register-3-inputs.png)

5. When it looks good, press Register Algorithm at the bottom of the page. A few seconds later you should see a modal dialog with a link to the algorithm registration process. 
![Register Algorithm submitted](_static/register-4-modal.png)

6. If you open that link in a new page or tab, you can monitor the progress of registration and see any error messages. Here is an example error message:

![Register Algorithm error](_static/register-5-status.png)

If the process continues without failing (this may take some time) you will ultimately see "Job succeeded":
![Register Algorithm success](_static/register-6-success.png)

7. Now that the algorithm has registered, you will see it in the View & Submit Jobs tool in the Algorithm drop-down menu. It will be labeled with the name you put into the Algorithm Name field in the registration form you just submitted.

## Run the Algorithm as a Job and Monitor it

### The Jobs UI
MAAP is configured to run up to 4,000 concurrent jobs. There are two additional ways to run a job: via the Jobs UI in the Launcher, or via a call to the maap-py Python library.

The Jobs UI will let you run and monitor jobs easily. You can find full documentation in the system reference guide for [the Jobs UI](../system_reference_guide/ade_custom_extensions/jobsui.ipynb).
You can also find specific documentation on [how to submit jobs](../system_reference_guide/faq/job_submit.ipynb) and [how to monitor jobs](../system_reference_guide/faq/job_check_status.ipynb) in the System Reference Guide FAQs.

![job_ui_access](../_static/jobs_ui_access.png)

Some alternative methods of running the job are found below.

### Pop-up
- Click DPS/MAS Operations menu -> Execute DPS Job
- Select your algorithm from the dropdown
- A new popup will ask for inputs; if it doesn’t take inputs, the popup will say so.
- Click OK again to view the ID for the job just submitted.

OR

### maap-py
Import the `maap-py` library: if in Jupyter, click the small blue MAAP button in the top left corner to automatically insert code. If using a script, add these lines manually at the top of your notebook: 
```
from maap.maap import MAAP
maap = MAAP()
```

Pass your algorithm’s name, version, required inputs, and username to the function `maap.submitJob (identifier is job- algo_name:algo_version)`
Check result: `maap.getJobResult()`
